In [ ]:
import pandas as pd
import os
import shutil
from google.colab import drive
from pathlib import Path
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
SAVE_CSV_PATH = '/content/drive/MyDrive/HECTO/Dataset/CSV/FF++_final.csv'
ZIP_PATH = "/content/drive/MyDrive/HECTO/Dataset/final_files/FF_Frames_Final.zip"
LOCAL_EXTRACT_BASE = Path("/content/FF++_Extract")

In [ ]:
if LOCAL_EXTRACT_BASE.exists():
    shutil.rmtree(LOCAL_EXTRACT_BASE)
LOCAL_EXTRACT_BASE.mkdir(parents=True, exist_ok=True)

print(f"🔓 압축 해제 시작: {ZIP_PATH}")
# -q: quiet, -d: target directory
!unzip -q "{ZIP_PATH}" -d "{LOCAL_EXTRACT_BASE}"
print("✅ 압축 해제 완료!")

FF_FRAMES_LOCAL = LOCAL_EXTRACT_BASE / "FF++_frames"

data_list = []
print("🔍 로컬 폴더 스캔 중 (Real=1, Fake=0)...")

🔓 압축 해제 시작: /content/drive/MyDrive/HECTO/Dataset/final_files/FF_Frames_Final.zip
✅ 압축 해제 완료!
🔍 로컬 폴더 스캔 중 (Real=1, Fake=0)...


In [ ]:
# 1. 데이터 수집
data_list = []
# 사용자가 설정한 LOCAL_EXTRACT_BASE 내에서 모든 jpg를 찾습니다.
print(f"🔍 스캔 시작: {LOCAL_EXTRACT_BASE}")

all_images = list(LOCAL_EXTRACT_BASE.rglob("*.jpg"))

if not all_images:
    print("❌ 파일을 하나도 찾지 못했습니다. 압축 해제 경로를 확인하세요!")
    !ls -R {LOCAL_EXTRACT_BASE} | head -n 15
else:
    for img_path in all_images:
        path_str = str(img_path).lower()

        # 라벨링 (FF++ 특성 반영)
        label = 1 if 'real' in path_str or 'youtube' in path_str else 0

        # Manipulation Method 추출
        # FF++_frames/Deepfakes/000_111/frame_0.jpg 구조라고 가정 시
        # 뒤에서 3번째 부모 폴더가 조작 방식임
        parent_folders = img_path.parts
        manip_method = parent_folders[-3] if label == 0 else 'original'

        # Video ID (이미지가 든 바로 위 폴더명)
        video_id = img_path.parent.name

        # Base ID (앞 3자리 숫자) - 000_111 -> 000
        base_id = video_id[:3]

        data_list.append({
            'video_id': video_id,
            'frame_path': str(img_path),
            'label': label,
            'source': 'FF++',
            'manipulation_method': manip_method,
            'base_id': base_id
        })

    df = pd.DataFrame(data_list)

    # 데이터가 정상적으로 수집되었을 때만 분할 진행
    if not df.empty:
        # 2. 8:1:1 원본 ID 기준 '랜덤' 분할
        unique_bases = pd.Series(df['base_id'].unique())

        print(f"✅ 총 {len(unique_bases)}개의 배경 그룹을 무작위로 섞습니다.")

        # 무작위 분할 (8:1:1)
        train_b, temp_b = train_test_split(unique_bases, test_size=0.2, random_state=42)
        val_b, test_b = train_test_split(temp_b, test_size=0.5, random_state=42)

        # Split 마킹
        df['split'] = None
        df.loc[df['base_id'].isin(train_b), 'split'] = 'train'
        df.loc[df['base_id'].isin(val_b), 'split'] = 'val'
        df.loc[df['base_id'].isin(test_b), 'split'] = 'test'

        # celeb과 동일한 포맷을 위해 base_id 삭제
        df_final = df.drop(columns=['base_id'])

        # 결과 리포트
        print("\n--- [랜덤 분할 결과] ---")
        print(df_final.groupby(['split', 'label']).size().unstack(fill_value=0))

        # 저장
        os.makedirs(os.path.dirname(SAVE_CSV_PATH), exist_ok=True)
        df_final.to_csv(SAVE_CSV_PATH, index=False)
        print(f"\n🚀 CSV 저장 완료: {SAVE_CSV_PATH}")
    else:
        print("❌ 데이터프레임이 생성되지 않았습니다. 조건을 확인하세요.")

🔍 스캔 시작: /content/FF++_Extract
✅ 총 1000개의 배경 그룹을 무작위로 섞습니다.

--- [랜덤 분할 결과] ---
label      0      1
split              
test    1500   1500
train  12000  12000
val     1500   1500

🚀 CSV 저장 완료: /content/drive/MyDrive/HECTO/Dataset/CSV/FF++_final.csv
